In [1]:
# import dependencies
import numpy as np
import pandas as pd
import matplotlib as plt
import matplotlib.pyplot as plotty
import missingno as msno

from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor

# suppress ugly warning messages
# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline


# create a lambda to give days since
days_since = lambda x: x - df.time_stamp.min()
remove_whitespace = lambda x: x.replace(" ", "")

# Data Preparation

In [2]:
# price_paid               int64 [233500, 270000, 176000, 450000, 440000]
# money, possibly look into normalizing
# deed_date               object ['05/07/2017', '05/07/2017', '05/07/2017', '05/07/2017', '05/07/2017']
# date object convert to timestamp and then add days since value
# property_type           object ['F', 'T', 'O', 'S', 'D']
# categorical data perform label encoder
# new_build               object ['N', 'Y']
# categorical data perform label encoder
# estate_type             object ['L', 'F']
# categorical data perform label encoder
# district                object ['TOWER HAMLETS', 'CITY OF LONDON', 'HACKNEY', 'HARROW', 'WALTHAM FOREST']
# categorical data perform label encoder
# transaction_category    object ['A', 'B']
# categorical data perform label encoder

In [3]:
# Load the datasets
pricing_df = pd.read_csv('../data/raw/01_06_2014_until_04_06_2019.csv')
# We can load the 1gb master csv of postcode data into memory with pandas
postcode_df = pd.read_csv('../data/raw/NSPL_MAY_2019_UK.csv', low_memory=False)

In [4]:
pricing_df.postcode = pricing_df.postcode.astype(str).map(remove_whitespace)
postcode_df.pcd = postcode_df.pcd.astype(str).map(remove_whitespace)

In [5]:
# Merging the datasets to bring in lat and lon values.
left = pricing_df
right = postcode_df
df = pd.merge(left, right, how='left', left_on='postcode', right_on='pcd')
# Drop the rows that have NaN in lat or long column
df = df.dropna(subset=['lat'])
df = df.dropna(subset=['long'])
df.shape

(343135, 57)

In [6]:
# Split by Boroughs of interest
lewisham = df.loc[df['district'] == 'LEWISHAM']
croydon = df.loc[df['district'] == 'CROYDON']
kensington_chelsea = df.loc[df['district'] == 'KENSINGTON AND CHELSEA']
westminster = df.loc[df['district'] == 'CITY OF WESTMINSTER']

In [7]:
# Check for duplicate values
df.duplicated().any()

False

In [8]:
lewisham.shape

(20213, 57)

In [9]:
df = westminster
# remove property type other
df = df.loc[df['property_type'] != 'O']
# Remove outliers in price
df = df.loc[(df['price_paid'] < (1000000)) & (df['price_paid'] > (10000))]
# Convert the date object type to... timestamp
df['time_stamp'] = pd.to_datetime(df['deed_date'])
# Create features for year and month
df['year'] = df['time_stamp'].dt.year
df['month'] = df['time_stamp'].dt.month
# apply the lambda on time_stamp, writing to new column
df['days_since'] = df['time_stamp'].map(days_since)
# encode the 'days since' and write to new column
df['days_since'] = df['days_since'].dt.days
# Setting the index to timestamp for time series operations
df.index = df['time_stamp']
# Dropping unnecessary column
df.drop(['time_stamp'], axis=1)
# round to the nearest thousand £
df['price_paid'] = df['price_paid'].round(decimals=3)
# df.drop(['unique_id', 'paon', 'saon', 'linked_data_url', 'street', 'locality', 'county', 'town', 'postcode'], axis=1)
df['property_type_encoded'] = df[['property_type']].apply(preprocessing.LabelEncoder().fit_transform)
df['new_build_encoded'] = df[['new_build']].apply(preprocessing.LabelEncoder().fit_transform)
df['estate_type_encoded'] = df[['estate_type']].apply(preprocessing.LabelEncoder().fit_transform)
df['transaction_category_encoded'] = df[['transaction_category']].apply(preprocessing.LabelEncoder().fit_transform)
# Selecting columns for modelling
df = df[['price_paid', 'lat','long','year','month','days_since','property_type_encoded','new_build_encoded','estate_type_encoded','transaction_category_encoded']]
westminster_prep = df
westminster_prep.shape

(8224, 10)

In [10]:
df = kensington_chelsea
# remove property type other
df = df.loc[df['property_type'] != 'O']
# Remove outliers in price
df = df.loc[(df['price_paid'] < (1000000)) & (df['price_paid'] > (10000))]
# Convert the date object type to... timestamp
df['time_stamp'] = pd.to_datetime(df['deed_date'])
# Create features for year and month
df['year'] = df['time_stamp'].dt.year
df['month'] = df['time_stamp'].dt.month
# apply the lambda on time_stamp, writing to new column
df['days_since'] = df['time_stamp'].map(days_since)
# encode the 'days since' and write to new column
df['days_since'] = df['days_since'].dt.days
# Setting the index to timestamp for time series operations
df.index = df['time_stamp']
# Dropping unnecessary column
df.drop(['time_stamp'], axis=1)
# round to the nearest thousand £
df['price_paid'] = df['price_paid'].round(decimals=3)
# df.drop(['unique_id', 'paon', 'saon', 'linked_data_url', 'street', 'locality', 'county', 'town', 'postcode'], axis=1)
df['property_type_encoded'] = df[['property_type']].apply(preprocessing.LabelEncoder().fit_transform)
df['new_build_encoded'] = df[['new_build']].apply(preprocessing.LabelEncoder().fit_transform)
df['estate_type_encoded'] = df[['estate_type']].apply(preprocessing.LabelEncoder().fit_transform)
df['transaction_category_encoded'] = df[['transaction_category']].apply(preprocessing.LabelEncoder().fit_transform)
# Selecting columns for modelling
df = df[['price_paid', 'lat','long','year','month','days_since','property_type_encoded','new_build_encoded','estate_type_encoded','transaction_category_encoded']]
kensington_chelsea_prep = df
kensington_chelsea_prep.shape

(4193, 10)

In [11]:
df = croydon
# remove property type other
df = df.loc[df['property_type'] != 'O']
# Remove outliers in price
df = df.loc[(df['price_paid'] < (1000000)) & (df['price_paid'] > (10000))]
# Convert the date object type to... timestamp
df['time_stamp'] = pd.to_datetime(df['deed_date'])
# Create features for year and month
df['year'] = df['time_stamp'].dt.year
df['month'] = df['time_stamp'].dt.month
# apply the lambda on time_stamp, writing to new column
df['days_since'] = df['time_stamp'].map(days_since)
# encode the 'days since' and write to new column
df['days_since'] = df['days_since'].dt.days
# Setting the index to timestamp for time series operations
df.index = df['time_stamp']
# Dropping unnecessary column
df.drop(['time_stamp'], axis=1)
# round to the nearest thousand £
df['price_paid'] = df['price_paid'].round(decimals=3)
# df.drop(['unique_id', 'paon', 'saon', 'linked_data_url', 'street', 'locality', 'county', 'town', 'postcode'], axis=1)
df['property_type_encoded'] = df[['property_type']].apply(preprocessing.LabelEncoder().fit_transform)
df['new_build_encoded'] = df[['new_build']].apply(preprocessing.LabelEncoder().fit_transform)
df['estate_type_encoded'] = df[['estate_type']].apply(preprocessing.LabelEncoder().fit_transform)
df['transaction_category_encoded'] = df[['transaction_category']].apply(preprocessing.LabelEncoder().fit_transform)
# Selecting columns for modelling
df = df[['price_paid', 'lat','long','year','month','days_since','property_type_encoded','new_build_encoded','estate_type_encoded','transaction_category_encoded']]
croydon_prep = df
croydon_prep.shape

(5237, 10)

In [12]:
df = lewisham
# remove property type other
df = df.loc[df['property_type'] != 'O']
# Remove outliers in price
df = df.loc[(df['price_paid'] < (1000000)) & (df['price_paid'] > (10000))]
# Convert the date object type to... timestamp
df['time_stamp'] = pd.to_datetime(df['deed_date'])
# Create features for year and month
df['year'] = df['time_stamp'].dt.year
df['month'] = df['time_stamp'].dt.month
# apply the lambda on time_stamp, writing to new column
df['days_since'] = df['time_stamp'].map(days_since)
# encode the 'days since' and write to new column
df['days_since'] = df['days_since'].dt.days
# Setting the index to timestamp for time series operations
df.index = df['time_stamp']
# Dropping unnecessary column
df.drop(['time_stamp'], axis=1)
# round to the nearest thousand £
df['price_paid'] = df['price_paid'].round(decimals=3)
# df.drop(['unique_id', 'paon', 'saon', 'linked_data_url', 'street', 'locality', 'county', 'town', 'postcode'], axis=1)
df['property_type_encoded'] = df[['property_type']].apply(preprocessing.LabelEncoder().fit_transform)
df['new_build_encoded'] = df[['new_build']].apply(preprocessing.LabelEncoder().fit_transform)
df['estate_type_encoded'] = df[['estate_type']].apply(preprocessing.LabelEncoder().fit_transform)
df['transaction_category_encoded'] = df[['transaction_category']].apply(preprocessing.LabelEncoder().fit_transform)
# Selecting columns for modelling
df = df[['price_paid', 'lat','long','year','month','days_since','property_type_encoded','new_build_encoded','estate_type_encoded','transaction_category_encoded']]
lewisham_prep = df
lewisham_prep.shape

(19076, 10)

In [13]:
# df
# df['price_paid','property_type','new_build', 'estate_type','transaction_category',
#        'lat', 'long', 'time_stamp', 'year', 'month', 'days_since',
#     'property_type_encoded', 'new_build_encoded',
#        'estate_type_encoded', 'transaction_category_encoded']

In [14]:
# dfs = [lewisham_prep, kensington_chelsea_prep, westminster_prep, croydon_prep]
# results = []

# for d in dfs:
#     shape = d.shape
#     results.append(shape)

# print(results)

In [15]:
# df = lewisham_prep.sample(1000, random_state=1)

In [19]:
# df = df[['price_paid', 'lat','long','year','month','days_since','property_type_encoded','new_build_encoded','estate_type_encoded','transaction_category_encoded']]
# df.dropna()
# df.isna().any()
df = lewisham_prep
df.to_pickle('../data/processed/lewisham_prep.pkl')
# df = kensington_chelsea_prep
# df = croydon_prep
# df = westminster_prep
# df
# corr = d.corr()
# corr.style.background_gradient(cmap='coolwarm')

In [17]:
# subset - district days since time series latlon outliers removed
df

,price_paid,lat,long,year,month,days_since,property_type_encoded,new_build_encoded,estate_type_encoded,transaction_category_encoded
time_stamp,,,,,,,,,,
2018-02-11,434995,51.532429,0.039803,2018,2,1496,1,1,1,0
2017-08-09,371000,51.472269,-0.011132,2017,8,1310,1,0,1,0
2016-05-02,350000,51.472269,-0.011132,2016,5,846,1,0,1,0
2015-07-13,290000,51.472269,-0.011132,2015,7,552,1,0,1,0
2018-02-03,620000,51.472269,-0.011132,2018,2,1488,1,0,1,0
2016-01-22,592000,51.472269,-0.011132,2016,1,745,1,0,1,0
2016-11-04,387500,51.472307,-0.010728,2016,11,1032,1,0,1,0
2017-01-12,415000,51.472307,-0.010728,2017,1,1101,1,0,1,0
2017-03-30,450000,51.472307,-0.010728,2017,3,1178,1,0,1,0


# Comparing Various Models

In [27]:
df = pd.read_pickle('../data/processed/lewisham_prep.pkl')
# Split data into predictors X and output Y
predictors = ['lat','long','year','month','days_since','property_type_encoded','new_build_encoded','estate_type_encoded','transaction_category_encoded']
X = df[predictors]
y = df['price_paid']
# Split 80/20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# prepare models
# df = unpickled
models = []

models.append(('LR', LinearRegression()))
# models.append(('PER', Perceptron()))
# models.append(('LA', Lasso()))
# models.append(('DTR', DecisionTreeRegressor()))
models.append(('RFR', RandomForestRegressor(n_estimators=40, n_jobs=-1)))
# models.append(('GPR', GaussianProcessRegressor()))
# models.append(('SVR', SVR()))
# models.append(('ABR', AdaBoostRegressor()))

# results = []
names = []

# evaluate each model in turn
for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = model.score(X_test, y_test)
    # Evaluate the performance of the algorithm with metrics
    # Mean Absolute Error
    mae = metrics.mean_absolute_error(y_test, y_pred)
    # Mean Squared Error
    mse = metrics.mean_squared_error(y_test, y_pred)
    # Root Mean Squared Error
    rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    msg = "%s: %f, mae: %f, mse: %f, rmse: %f" % (name, score, mae, mse, rmse)
    print(msg)

# # boxplot algorithm comparison ##
# fig = plotty.figure()
# fig.suptitle('Algorithm Comparison')
# ax = fig.add_subplot(111)
# plotty.boxplot(results)
# ax.set_xticklabels(names)
# plotty.show()

LR: 0.454236, mae: 91912.923428, mse: 14406435857.447372, rmse: 120026.813077
RFR: 0.652582, mae: 69387.080637, mse: 9170741647.881842, rmse: 95763.989306
